<a href="https://colab.research.google.com/github/bacchus00/trabajo-dirigido/blob/main/rdfing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib
!pip install unidecode
!pip install validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=6560dee4e6a6f2ea70a3bb230ecc6c11254a6e6b6f447843e1107096a6045093
  Stored in directory: /root/.cache/pip/wheels/19/09/72/3eb74d236bb48bd0f3c6c3c83e4e0c5bbfcbcad7c6c3539db8
Successfully built validators


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import validators as val
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import urllib.parse
from unidecode import unidecode

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Bushe/Trabajo Dirigido/cleaner_data.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9,10,14,17,18,22,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
pd.set_option("display.max_rows", None)
print(df['Descripcion1'].unique())

['Reembolso solicitado candidato o partido' 'No definido'
 'Aporte reservado' 'Aporte anónimo' 'Aporte patrimonial' 'Aporte público'
 'Anticipo financiamiento fiscal'
 'Reembolso facturas cedidas partido politico' 'Aporte propio' nan
 'Crédito institución financiera'
 'Intereses crédito institución financiera'
 'Aporte público partido politico' 'Aporte trabajo voluntario'
 'Reembolsos facturas endosadas' 'Aporte por Usufructo'
 'Aportes en dinero recibido por candidato de un partido politico'
 'Aportes Reservado' 'Aporte personal del candidato' 'Aportes Anónimos'
 'Intereses devengados por créditos con Instituciones Financieras'
 'Aporte por trabajo voluntario' 'Créditos con Instituciones Financieras'
 'Reembolso solicitado por cesión de derecho a partido político'
 'Aportes Privados de Carácter Público' 'Usufructo'
 'Reembolsos solicitado por endosos' 'Aporte Propio'
 'Aportes Privados de Carácter Público de Partidos Politicos'
 'Intereses por crédito con Institución Financiera' 'APOR

In [ ]:
# list of ingresos that actually are contributions
ingresos_def = [
  'No definido'
  'Aporte reservado'
  'Aporte anónimo'
  'Aporte patrimonial'
  'Aporte público'
  'Aporte propio'
  'Aporte público partido politico',
  'Aporte trabajo voluntario',
  'Aporte por Usufructo',
  'Aportes en dinero recibido por candidato de un partido politico',
  'Aportes Reservado',
  'Aporte personal del candidato',
  'Aportes Anónimos',
  'Aporte por trabajo voluntario',
  'Aportes Privados de Carácter Público',
  'Usufructo',
  'Aporte Propio',
  'Aportes Privados de Carácter Público de Partidos Politicos',
  'APORTE PROPIO',
  'Aportes de personas naturales',
  'Aportes en especies recibido por candidato de un partido politico',
  'Aporte estimado por uso de un vehículo de un tercero',
  'Aporte estimado por propaganda en espacio privado',
  'Aporte estimado por servicios profesionales de un tercero',
  'Aportes menores sin publicidad',
  'Aporte estimado en especies (mantención y colaciones)',
  'Aporte personal del candidato (vehículo, propiedades entre otros)',
  'Aporte estimado por uso de propiedad de un tercero' 'Comodato',
  'Aporte estimado por propaganda en medios de comunicación digital',
  'Aporte estimado por propaganda en radioemisoras',
  'Aporte estimado por propaganda escrita',
  'Donaciones',
  'Mutuos',
  'Otros actos o contratos a título gratuito',
  'Aportes propios de partido politico',
  'Aportes en especies o servicios de terceros',
  'Aporte con  Publicidad',
  'Aportes en dinero de Partidos Políticos',
  'Aporte propio en especies o servicios',
  'Aportes en especies o servicios de partidos politicos',
  'Aporte propio en efectivo del candidato',
  'Aportes sin Publicidad',
  'Financiamiento a partidos politicos',
  'Aportes con publicidad',
  'Aportes sin publicidad',
  'Aporte Propio por Imputacion de reembolso a candidatos',
  'Aportes Partido Politico',
  'Aporte Propio por Imputacion de reembolso a instituciones financieras',
  'Financiamiento a candidatos independientes',
  'Cesion de derecho a reembolso',
  'Imputación de gasto electoral de primarias',
  'Imputación saldo de financiamiento primarias',
  'Imputación de gasto presunto de partidos políticos',
  'Aportes de Partido politico',
  'Aportes con Publicidad',
  'Aportes Partidos politicos']

In [ ]:
# filtering only contributions
aportes = df[df['Descripcion1'].isin(ingresos_def)]
len(aportes.index)

360168

In [ ]:
aportes = aportes

In [ ]:
# namespaces
ctbr = Namespace("http://example.org/contributor/")
ctb = Namespace("http://example.org/contribution/")
prty = Namespace("http://example.org/party/")
elec = Namespace("http://example.org/election/")
cnd = Namespace("http://example.org/candidate/")
trry = Namespace("http://example.org/territory/")

#custom properties
rut = URIRef(Namespace("http://example.org/my-schema/rut/"))
contributed = URIRef(Namespace("http://example.org/my-schema/contributed/"))
receivedBy = URIRef(Namespace("http://example.org/my-schema/receivedBy/"))
forElection = URIRef(Namespace("http://example.org/my-schema/forElection/"))
year = URIRef(Namespace("http://example.org/my-schema/year/"))
amountCLP = URIRef(Namespace("http://example.org/my-schema/amountCLP/"))

#predefined properties
schema = Namespace('http://schema.org/')

In [ ]:
# function to transform strings to form valid iris
def iri_str(x):
  a_string = unidecode(x)
  a_string = a_string.replace(' ','-')
  a_string = a_string.replace('Ñ','N')
  for c in ['%', '{', '}', '<', '>','|','"', ':']:
    a_string = a_string.replace(c,'') 
  return a_string

In [ ]:
g = Graph()

for index, row in aportes.iterrows():
  #check for nans
  has_contributor = row['Aportante']==row['Aportante']
  has_rut_contributor = row['RUTAportante']==row['RUTAportante']
  has_party = row['Partido']==row['Partido']
  has_rut_party = row['RUTPartido']==row['RUTPartido']
  has_candidate = row['Candidato']==row['Candidato']
  has_rut_candidate = row['RUTCandidato']==row['RUTCandidato']
  has_date = row['Fecha']==row['Fecha']
  has_amount = row['Monto']==row['Monto']

  #add contributor
  giver = ''
  if has_contributor:
    contributor = URIRef(ctbr+iri_str(row['Aportante']))
    giver = row['Aportante']
    #g.add((contributor, RDF.type, FOAF.Person))
    g.add((contributor, URIRef(schema+'name'), Literal(row['Aportante'], datatype=XSD.string)))
    if has_rut_contributor:
      g.add((contributor, rut, Literal(row['RUTAportante'], datatype=XSD.string)))

  #add party
  if has_party:
    party = URIRef(prty+iri_str(row['Partido']))
    reciever = row['Partido']
    reciever_iri = party
    g.add((party, URIRef(schema+'name'), Literal(row['Partido'], datatype=XSD.string)))
    if has_rut_party:
      g.add((party, rut, Literal(row['RUTPartido'], datatype=XSD.string)))

  #add candidate
  if has_candidate:
    candidate = URIRef(cnd+iri_str(row['Candidato']))
    reciever = row['Candidato']
    reciever_iri = candidate
    g.add((candidate, RDF.type, FOAF.Person))
    g.add((candidate, URIRef(schema+'name'), Literal(row['Candidato'], datatype=XSD.string)))
    if has_rut_candidate:
      g.add((candidate, rut, Literal(row['RUTCandidato'], datatype=XSD.string)))
    if has_party:
      g.add((candidate, URIRef(schema+'memberOf'), party))


  #add election
  election = URIRef(elec+iri_str(row['Eleccion']))
  g.add((election, year, Literal(row['Archivo'].split('-')[0], datatype=XSD.gYear)))
  territories = [
    'TerritorioElectoral', 
    'Comuna',
    'Region', 
    'Distrito',
    'Circunscripcion', 
    'CircunscripcionSenatorial',
    'CircunscripcionProvincial']
  for territory in territories:
    if row[territory] == row[territory]:
      g.add((election, URIRef(schema+'location'), Literal(row[territory], datatype=XSD.string)))
    
  #add contribution
  contribution = URIRef(ctb + iri_str(giver) +'-a-' + iri_str(reciever) + '-' + str(row['Fecha'])+'-'+str(index))
  if has_amount:
    g.add((contribution, amountCLP, Literal(int(row['Monto']), datatype=XSD.int)))
  if has_date:
    g.add((contribution, URIRef(schema+'datePosted'), Literal(row['Fecha'], datatype=XSD.date)))
  if has_contributor:
    g.add((contributor, contributed, contribution))
  g.add((contribution, receivedBy, reciever_iri))
  g.add((contribution, forElection, election))

In [ ]:
# saving results to ttl
g.serialize(destination="/content/drive/MyDrive/Bushe/Trabajo Dirigido/aportes.ttl")

<Graph identifier=N8ed0f9a55ae846b1bd3dc85ea3fb7b48 (<class 'rdflib.graph.Graph'>)>